<a href="https://colab.research.google.com/github/kvnmnnng/DS-Unit-2-Kaggle-Challenge/blob/master/DS_module_project_221_Kevin_Manning_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [ ]:
# see if changes are saved (12)

In [ ]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [ ]:
import pandas as pd

df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
              pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))
# YOUR CODE HERE
df = df.set_index('id')

df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [ ]:
df.shape

(59400, 40)

**Test 2**

In [ ]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [ ]:
target = 'status_group'

# YOUR CODE HERE
df_features= df.drop(columns= 'status_group')
X= df_features
y= df[target]
X.shape, y.shape

((59400, 39), (59400,))

**Test 3**

In [ ]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [ ]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
train, val= train_test_split(df, train_size= 0.8, test_size= 0.2,
                             stratify= df['status_group'], random_state= 41)
train.shape, val.shape

((47520, 40), (11880, 40))

In [ ]:
X_train= train.drop(columns= 'status_group')
X_val= val.drop(columns= 'status_group')
y_train= train['status_group']
y_val= val['status_group']

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 39), (11880, 39), (47520,), (11880,))

**Test 4**

In [ ]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [ ]:
# YOUR CODE HERE
baseline_acc= train['status_group'].value_counts(normalize= True).functional
baseline_acc
#print('Baseline Accuracy:', baseline_acc)

0.5430765993265994

**Task 5**

In [ ]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [ ]:
X_train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35860,0.0,2011-08-03,Kkkt,0,KKKT,32.801778,-8.938634,Mshane,0,Lake Rukwa,Nzije A,Mbeya,12,6,Mbozi,Halungu,0,False,GeoData Consultants Ltd,VWC,NaN,False,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,seasonal,seasonal,shallow well,shallow well,groundwater,hand pump,hand pump
69373,50.0,2011-03-04,Private,-43,Priva,39.178942,-8.140355,Salehe Muhameid Salehe,0,Rufiji,Sunga Pwani,Pwani,60,53,Rufiji,Chumbi,1,True,GeoData Consultants Ltd,Private operator,NaN,True,2005,nira/tanira,nira/tanira,handpump,private operator,commercial,pay per bucket,per bucket,soft,good,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump
15849,0.0,2011-02-16,Schoo,1921,Schoo,35.773879,-8.123505,none,0,Rufiji,Msegera,Iringa,11,7,Kilolo,Ukumbi,700,True,GeoData Consultants Ltd,VWC,NaN,True,2007,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
3085,0.0,2013-02-25,Netherlands,0,DWE,34.008617,-3.113730,Buzinza,0,Lake Victoria,Mwabeya,Shinyanga,17,2,Maswa,Ipililo,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,swn 80,swn 80,handpump,wug,user-group,other,other,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
22155,0.0,2013-12-03,World Vision,1495,World Vision,37.921105,-4.180571,Kwa Haji,0,Pangani,Mkongo,Kilimanjaro,3,3,Same,Msindo,200,True,GeoData Consultants Ltd,Water authority,Nakombo,True,1995,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe


In [ ]:
X_train.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [ ]:
X_val.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [ ]:
# clean the data
import numpy as np

def wrangle(X):
  X= X.copy()
  X['latitude']= X['latitude'].replace(-2e-08, 0)
  cols_with_zeros= ['longitude', 'latitude']
  for col in cols_with_zeros:
    X[col]= X[col].replace(0, np.nan)

  X= X.drop(columns= 'quantity_group')

  return X


X_train= wrangle(X_train)
X_val= wrangle(X_val)

In [ ]:
X_train.shape, X_val.shape

((47520, 38), (11880, 38))

In [ ]:
# further clean up the data
train_features = X_train.copy()

numeric_features= train_features.select_dtypes(include= 'number').columns.tolist()
cardinality= train_features.select_dtypes(exclude= 'number').nunique()
categorical_features= cardinality[cardinality <= 50].index.tolist()
features= numeric_features + categorical_features
print(features)

# need to make this a function!

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [ ]:
X_train= X_train[features]
X_val= X_val[features]

In [ ]:
X_train.shape, X_val.shape

((47520, 30), (11880, 30))

In [ ]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce

# YOUR CODE HERE
model = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier(min_samples_leaf= 20, random_state=34)
)

model.fit(X_train, y_train)

print('Train Accuracy', model.score(X_train, y_train))
print('Validation Accuracy', model.score(X_val, y_val))

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Train Accuracy 0.8061868686868687
Validation Accuracy 0.7686868686868686


**Test 6**

In [ ]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [ ]:
# YOUR CODE HERE
training_acc= model.score(X_train, y_train)
validation_acc= model.score(X_val, y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.8061868686868687
Validation Accuracy: 0.7686868686868686


**Test 7**

In [ ]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [ ]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')
# YOUR CODE HERE
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')
# YOUR CODE HERE
X_test= wrangle(X_test)
test_features = X_test.copy()

numeric_features= test_features.select_dtypes(include= 'number').columns.tolist()
cardinality= test_features.select_dtypes(exclude= 'number').nunique()
categorical_features= cardinality[cardinality <= 50].index.tolist()
features2= numeric_features + categorical_features
print(features2)

#submission.head()

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [ ]:
X_test

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,basin,region,public_meeting,recorded_by,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,1996,35.290799,-4.059696,0,21,3,321,2012,Internal,Manyara,True,GeoData Consultants Ltd,Parastatal,True,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
51630,0.0,1569,36.656709,-3.309214,0,2,2,300,2000,Pangani,Arusha,True,GeoData Consultants Ltd,VWC,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
17168,0.0,1567,34.767863,-5.004344,0,13,2,500,2010,Internal,Singida,True,GeoData Consultants Ltd,VWC,NaN,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
45559,0.0,267,38.058046,-9.418672,0,80,43,250,1987,Ruvuma / Southern Coast,Lindi,NaN,GeoData Consultants Ltd,VWC,True,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,shallow well,shallow well,groundwater,other,other
49871,500.0,1260,35.006123,-10.950412,0,10,3,60,2000,Ruvuma / Southern Coast,Ruvuma,NaN,GeoData Consultants Ltd,Water Board,True,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39307,0.0,34,38.852669,-6.582841,0,6,1,20,1988,Wami / Ruvu,Pwani,True,GeoData Consultants Ltd,VWC,True,mono,mono,motorpump,vwc,user-group,never pay,never pay,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe
18990,1000.0,0,37.451633,-5.350428,0,4,7,2960,1994,Pangani,Tanga,True,GeoData Consultants Ltd,VWC,False,nira/tanira,nira/tanira,handpump,vwc,user-group,pay annually,annually,salty,salty,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
28749,0.0,1476,34.739804,-4.585587,0,13,2,200,2010,Internal,Singida,True,GeoData Consultants Ltd,VWC,NaN,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,dam,dam,surface,communal standpipe,communal standpipe


In [ ]:
X_test= X_test[features2]
X_test.shape

(14358, 30)

In [ ]:
model = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier(min_samples_leaf= 20, random_state=34)
)

model.fit(X_train, y_train)

print('Train Accuracy', model.score(X_train, y_train))
print('Validation Accuracy', model.score(X_val, y_val))

y_pred = model.predict(X_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Train Accuracy 0.8061868686868687
Validation Accuracy 0.7686868686868686


In [ ]:
submission= pd.DataFrame(y_pred, columns= ['status_group'],  index= X_test.index)

In [ ]:
submission.reset_index(inplace= True)

In [ ]:
submission.rename(columns= {'index':'id'})

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14353,39307,non functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [ ]:
submission.shape

(14358, 2)

In [ ]:
submission.columns

Index(['id', 'status_group'], dtype='object')

In [ ]:
X_test

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,basin,region,public_meeting,recorded_by,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,1996,35.290799,-4.059696,0,21,3,321,2012,Internal,Manyara,True,GeoData Consultants Ltd,Parastatal,True,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
51630,0.0,1569,36.656709,-3.309214,0,2,2,300,2000,Pangani,Arusha,True,GeoData Consultants Ltd,VWC,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
17168,0.0,1567,34.767863,-5.004344,0,13,2,500,2010,Internal,Singida,True,GeoData Consultants Ltd,VWC,NaN,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
45559,0.0,267,38.058046,-9.418672,0,80,43,250,1987,Ruvuma / Southern Coast,Lindi,NaN,GeoData Consultants Ltd,VWC,True,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,shallow well,shallow well,groundwater,other,other
49871,500.0,1260,35.006123,-10.950412,0,10,3,60,2000,Ruvuma / Southern Coast,Ruvuma,NaN,GeoData Consultants Ltd,Water Board,True,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39307,0.0,34,38.852669,-6.582841,0,6,1,20,1988,Wami / Ruvu,Pwani,True,GeoData Consultants Ltd,VWC,True,mono,mono,motorpump,vwc,user-group,never pay,never pay,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe
18990,1000.0,0,37.451633,-5.350428,0,4,7,2960,1994,Pangani,Tanga,True,GeoData Consultants Ltd,VWC,False,nira/tanira,nira/tanira,handpump,vwc,user-group,pay annually,annually,salty,salty,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
28749,0.0,1476,34.739804,-4.585587,0,13,2,200,2010,Internal,Singida,True,GeoData Consultants Ltd,VWC,NaN,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,dam,dam,surface,communal standpipe,communal standpipe


In [ ]:
X_test.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year',
       'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management',
       'permit', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity', 'source',
       'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

**Test 8**

In [ ]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [ ]:
# YOUR CODE HERE
submission.to_csv('submission.csv', index=False)

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE
